In [2]:
import numpy as np
import pandas as pd
import ast

In [4]:
!ls csv_files

yelp_academic_dataset_business.csv  yelp_academic_dataset_tip_transposed.csv
yelp_academic_dataset_review.csv    yelp_academic_dataset_user.csv
yelp_academic_dataset_tip.csv


In [5]:
generic = lambda x: ast.literal_eval(x)
conv = {'friends': generic}

In [120]:
business = pd.read_csv('csv_files/yelp_academic_dataset_business.csv')
review = pd.read_csv('csv_files/yelp_academic_dataset_review.csv')
tip = pd.read_csv('csv_files/yelp_academic_dataset_tip_transposed.csv')
user = pd.read_csv('csv_files/yelp_academic_dataset_user.csv',converters=conv)

# Parse user

In [9]:
user_ids = user['user_id'].reset_index().set_index('user_id')['index'].to_dict()

In [10]:
user_table = user

In [11]:
friends = user['friends'].map(lambda friends: list(map(lambda x: user_ids[x],friends)))

In [12]:
elite = user[user['elite'].map(lambda x:type(x))==type("")]['elite'].map(lambda e:list(map(lambda x:int(x),e.split(","))))

In [17]:
friends.head()

0    [291, 126238, 272956, 30953, 18897, 1391, 2805...
1    [293790, 578622, 205428, 607172, 586444, 58459...
2    [47612, 111729, 458446, 54552, 6553, 19363, 12...
3    [589226, 495880, 434833, 769827, 117697, 19492...
4    [26967, 431106, 354308, 48663, 1556, 215429, 3...
Name: friends, dtype: object

# Parse business

In [121]:
business_ids = business['business_id'].reset_index().set_index('business_id')['index'].to_dict()

# Parse review

In [116]:
review_ids = review['review_id'].reset_index().set_index('review_id')['index'].to_dict()

In [88]:
review[review.user_id.isin(user_ids.keys())].user_id.map(lambda x: user_ids[x])

0          65412
1           8026
2          11466
3          29615
4          21931
           ...  
918675    270237
918676     90048
918677    333785
918678     80120
918679       457
Name: user_id, Length: 918678, dtype: int64

In [133]:
excluded = review[(review.user_id.isin(user_ids.keys()) & review.business_id.isin(business_ids.keys())).map(lambda x:not x)]

In [135]:
included = review[review.user_id.isin(user_ids.keys()) & review.business_id.isin(business_ids.keys())]

In [136]:
excluded

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
199241,41b2SLmjLcxTGLVRxASiDA,0,2006-04-11 09:05:18,0.0,WWYQ1ce6mNt7AvRHu8w-jQ,3.0,Rating purely on food and 18th hole view: 5 stars,NaN,NaN
199242,NaN,1,53bZ_EsXH71L7iFs5MP9_w,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
included.loc[:,'user_id'] = included.apply(lambda x: user_ids[x.user_id], axis=1)

/home/bastien/.local/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [138]:
included.loc[:,'business_id'] = included.apply(lambda x: business_ids[x.business_id], axis=1)

In [117]:
review_table = included

In [139]:
included.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,107044,0,2016-11-09 20:09:03,0.0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3.0,65412
1,30137,0,2018-01-30 23:07:38,0.0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7.0,8026
2,112122,5,2016-05-07 01:21:02,4.0,G7XHMxG0bx9oBJNECG4IFg,3.0,Tracy dessert had a big name in Hong Kong and ...,5.0,11466
3,87447,0,2017-12-15 23:27:08,1.0,-I5umRTkhw15RqpKMl_o1Q,1.0,"Walked in around 4 on a Friday afternoon, we s...",0.0,29615
4,15682,0,2014-12-17 19:04:33,0.0,0AsmPiAQduxh5jE_si8cLA,5.0,ended up here because Raku was closed and it r...,0.0,21931
